In [10]:
import os
import settings
import pandas as pd
import operator

In [2]:
from sklearn.tree import DecisionTreeRegressor


In [3]:
def read_data():
    df = pd.read_excel(os.path.join(settings.PROCESSED_DIR, "all_with_liwc_segmented.xls"), encoding="ISO-8859-1")
    return df

In [4]:
def sort_important_features(df):
    dt = DecisionTreeRegressor()
    predictors = df.columns.tolist()
    predictors = [p for p in predictors if p not in settings.NON_PREDICTORS]
    dt.fit(df[predictors], df[settings.TARGET])
    predictions = dt.predict(df[predictors])
    results = {name: score for name, score in zip(predictors, dt.feature_importances_)}
    sorted_results = sorted(results.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_results
    accuracy = dt.score(df[predictors], df[settings.TARGET])
    print("Accuracy: {}".format(accuracy))

In [5]:
df = read_data()

In [11]:
sort_important_features(df)

[('assent', 0.05225767963605709),
 ('i', 0.042376823651275118),
 ('filler', 0.031467634427374415),
 ('feel', 0.031343415532560731),
 ('home', 0.030178835316116395),
 ('anx_4q', 0.021306307999205531),
 ('hear', 0.021024205098571898),
 ('FairnessVirtue', 0.02091321047722362),
 ('tentat', 0.018341404482819925),
 ('posemo_2h', 0.017584855730968525),
 ('negate', 0.017301003309879603),
 ('negemo_3q', 0.017289327252231042),
 ('insight', 0.017118908518346992),
 ('achieve', 0.015989124391771588),
 ('leisure', 0.015920345326961682),
 ('work', 0.015560985314474298),
 ('Fairness', 0.014746037316736018),
 ('motion', 0.014624994611690019),
 ('adj', 0.014567834974902895),
 ('Authentic', 0.01455444799946511),
 ('AuthorityVirtue', 0.014378620349809671),
 ('number', 0.013869377090425709),
 ('verb', 0.013736955059467407),
 ('affect_3q', 0.013528102518499306),
 ('affect_2q', 0.013457668592055362),
 ('drives', 0.013408098107779563),
 ('prep', 0.013354331141989362),
 ('informal', 0.012790277624366426),
 ('s

In [12]:
sort_important_features(df)

[('assent', 0.05374759023299442),
 ('i', 0.040413810362394921),
 ('filler', 0.0353029689332717),
 ('feel', 0.030992686829931698),
 ('achieve', 0.024450556449488063),
 ('anx_4q', 0.023687028866467834),
 ('home', 0.023415156883918847),
 ('insight', 0.022622697394005076),
 ('hear', 0.021537359560209426),
 ('posemo_2h', 0.020358783370564081),
 ('Authentic', 0.020072119836371489),
 ('negate', 0.018754961799262376),
 ('posemo', 0.018420161003453887),
 ('tentat', 0.017797795158701033),
 ('leisure', 0.016653285899899707),
 ('negemo_3q', 0.016569141011485974),
 ('Fairness', 0.016168102964729831),
 ('negemo_4q', 0.013931949115355554),
 ('FairnessVirtue', 0.013915585590699784),
 ('motion', 0.013775226473864657),
 ('verb', 0.01352492358897134),
 ('affect_2q', 0.013283711137171254),
 ('drives', 0.013101028427934321),
 ('prep', 0.012970600572547829),
 ('adj', 0.012904695208193047),
 ('they', 0.012762897265978788),
 ('social', 0.012575178409317815),
 ('AuthorityVice', 0.012482341830080958),
 ('anger_

In [13]:
sort_important_features(df)

[('assent', 0.053446373532726299),
 ('i', 0.042966044727072376),
 ('filler', 0.031965993245469254),
 ('feel', 0.030649465660985122),
 ('home', 0.026411716742961579),
 ('hear', 0.022762575789593738),
 ('negemo_3q', 0.019146839864561321),
 ('anx_4q', 0.019044504456985065),
 ('insight', 0.018377728673914538),
 ('leisure', 0.017017164945552141),
 ('negate', 0.016735271757785882),
 ('posemo_2h', 0.016452591831682985),
 ('tentat', 0.016085480220518939),
 ('Fairness', 0.015482850489067501),
 ('achieve', 0.015451523002025448),
 ('work', 0.014873592433568315),
 ('motion', 0.014744243063363485),
 ('drives', 0.014151622693107225),
 ('prep', 0.014026873725640595),
 ('FairnessVirtue', 0.013991304096297832),
 ('number', 0.013773651930546566),
 ('Authentic', 0.013771593735002025),
 ('affect_1h', 0.013742409568221802),
 ('AuthorityVirtue', 0.013356311616328121),
 ('posemo', 0.013072000699123211),
 ('affect_2q', 0.01277659640888959),
 ('AuthorityVice', 0.012296831491365743),
 ('social', 0.0122026045317

## Random Forest

In [14]:
from sklearn.ensemble import RandomForestRegressor

In [15]:
def sort_important_features(df):
    rf = RandomForestRegressor()
    predictors = df.columns.tolist()
    predictors = [p for p in predictors if p not in settings.NON_PREDICTORS]
    rf.fit(df[predictors], df[settings.TARGET])
    predictions = rf.predict(df[predictors])
    results = {name: score for name, score in zip(predictors, rf.feature_importances_)}
    sorted_results = sorted(results.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_results
    accuracy = rf.score(df[predictors], df[settings.TARGET])
    print("Accuracy: {}".format(accuracy))

In [16]:
sort_important_features(df)

[('assent', 0.054343103705411309),
 ('i', 0.041598684277226128),
 ('filler', 0.019156318906792223),
 ('prep', 0.018915621224126367),
 ('tentat', 0.018814658359490161),
 ('motion', 0.015695550413957558),
 ('affiliation', 0.015445665131127755),
 ('number', 0.014170257208943413),
 ('hear', 0.014142964514268826),
 ('quant', 0.013319294152620606),
 ('insight', 0.01298988768552976),
 ('Fairness', 0.012533194104037831),
 ('posemo_2q', 0.012330331438558644),
 ('auxverb', 0.012293836444847623),
 ('ppron', 0.011862731746059518),
 ('drives', 0.011768556850593376),
 ('feel', 0.011479038641020472),
 ('cogproc', 0.011242461030777034),
 ('we', 0.011047640112710705),
 ('discrep', 0.010930559019624315),
 ('posemo_3q', 0.010847392175886242),
 ('negate', 0.010569718746544752),
 ('adj', 0.010540214003182084),
 ('differ', 0.010440474965772528),
 ('netspeak', 0.010419833187693649),
 ('posemo_change_h', 0.010250362185281478),
 ('conj', 0.010105463826912048),
 ('adverb', 0.010021714592524112),
 ('ipron', 0.00

In [17]:
sort_important_features(df)

[('i', 0.04801417849514495),
 ('assent', 0.043332302666767276),
 ('filler', 0.023866540427380935),
 ('informal', 0.017296802915982484),
 ('insight', 0.017236351455331499),
 ('MoralityGeneral', 0.016459677307924345),
 ('ppron', 0.016338199601523155),
 ('hear', 0.015685561174679452),
 ('drives', 0.015495170579094561),
 ('negate', 0.015030331024789576),
 ('family', 0.013884791618574779),
 ('Tone', 0.013360206936579458),
 ('we', 0.01277921050515116),
 ('feel', 0.011953754831806322),
 ('Authentic', 0.011546830908485285),
 ('interrog', 0.011426220050790244),
 ('relativ', 0.011394718768179149),
 ('adj', 0.011378717876290526),
 ('netspeak', 0.011133256292433811),
 ('leisure', 0.011127239897543176),
 ('anger_2q', 0.011092859925718242),
 ('sad_4q', 0.010794528952217903),
 ('motion', 0.010722371341341488),
 ('achieve', 0.010668759306815588),
 ('focuspresent', 0.01054036312578221),
 ('relig', 0.010375386522195124),
 ('conj', 0.010254726327981974),
 ('tentat', 0.010056508787143904),
 ('verb', 0.009

In [18]:
sort_important_features(df)

[('i', 0.04871616130944717),
 ('assent', 0.038140078956615267),
 ('filler', 0.024350454080533303),
 ('posemo', 0.018597047141343928),
 ('motion', 0.018557825037592613),
 ('informal', 0.017389682973338559),
 ('tentat', 0.017164824305062637),
 ('hear', 0.016621837938591129),
 ('female', 0.016393550856848561),
 ('posemo_2h', 0.014550812186301498),
 ('insight', 0.014296247507064378),
 ('relig', 0.012953404025010615),
 ('negate', 0.012026175031591519),
 ('Tone', 0.011703892953571538),
 ('number', 0.011526909513699448),
 ('we', 0.011440729155503935),
 ('adj', 0.011129969703688706),
 ('posemo_3q', 0.010771088635044567),
 ('work', 0.010581131513109222),
 ('affiliation', 0.010299441230497822),
 ('drives', 0.010247228567609968),
 ('power', 0.010050725946239455),
 ('conj', 0.009663311479286292),
 ('posemo_change_q', 0.0095057578882788516),
 ('home', 0.0094877973972217113),
 ('negemo_1q', 0.0094561865362911522),
 ('auxverb', 0.0093500730876919392),
 ('space', 0.009349980851555157),
 ('body', 0.009

## Linear Regression

In [20]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

/Users/owentemple/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [54]:
def create_summary(df):
    lr = LinearRegression()
    predictors = df.columns.tolist()
    predictors = ['assent', 'i', 'insight', 'filler', 'Fairness', 'posemo']
    lr.fit(df[predictors], df['applause'])

    X = df[predictors]
    X2 = sm.add_constant(X)
    est = sm.OLS(df['applause'], X2)
    est2 = est.fit()
    print(est2.summary())

In [55]:
create_summary(df)

                            OLS Regression Results                            
Dep. Variable:               applause   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     43.28
Date:                Thu, 11 Jan 2018   Prob (F-statistic):           2.03e-50
Time:                        18:22:43   Log-Likelihood:                -4851.0
No. Observations:                2406   AIC:                             9716.
Df Residuals:                    2399   BIC:                             9756.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.6590      0.140     11.864      0.0

In [64]:
def create_summary(df):
    lr = LinearRegression()
    predictors = df.columns.tolist()
    predictors = ['affect_change_h']
    lr.fit(df[predictors], df['applause'])

    X = df[predictors]
    X2 = sm.add_constant(X)
    est = sm.OLS(df['applause'], X2)
    est2 = est.fit()
    print(est2.summary())

In [65]:
create_summary(df)

                            OLS Regression Results                            
Dep. Variable:               applause   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     4.876
Date:                Thu, 11 Jan 2018   Prob (F-statistic):             0.0273
Time:                        18:41:22   Log-Likelihood:                -4972.2
No. Observations:                2406   AIC:                             9948.
Df Residuals:                    2404   BIC:                             9960.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.0525      0.041     